## Add SB1 geographic information.

In [1]:
import _utils
import _sb1_utils as sb1_utils
from shared_utils import utils
import geopandas as gpd
import numpy as np
import pandas as pd
from calitp_data_analysis.sql import to_snakecase

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(
/opt/conda/lib/python3.9/site-packages/fuzzywuzzy/fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning


In [2]:
import fuzzywuzzy
from fuzzywuzzy import process

In [3]:
pd.options.display.max_columns = 200
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

#### Non SHOPP Sample Projects
* Solutions for Congest Corridors (SCCP): 1
* Trade Corridor Enhancement Program (TCEP): 3
* Only 3 projects seem to have been awarded. 
    * east bay greenway multimodal corridor project phase 1
    * us 101woodside road interchange and port access project
    * watsonvillesanta cruz multimodal corridor program wscmcp cycle 3 project contract 1 sr 1 freedom to state park aux lanes bus on shoulders and coastal rail trail segment 12

In [4]:
nonshopp, nine_sample_projects = sb1_utils.load_nonshopp("cleaned_data_atp_tircp.xlsx")

/home/jovyan/data-analyses/project_prioritization/_sb1_utils.py:28: FutureWarning: The default value of regex will change from True to False in a future version.
/home/jovyan/data-analyses/project_prioritization/_sb1_utils.py:42: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.


In [5]:
# Load SB1
sb1 = sb1_utils.sb1_final()

/home/jovyan/data-analyses/project_prioritization/_sb1_utils.py:28: FutureWarning: The default value of regex will change from True to False in a future version.
/home/jovyan/data-analyses/project_prioritization/_sb1_utils.py:42: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
/home/jovyan/data-analyses/project_prioritization/_sb1_utils.py:48: FutureWarning: The default value of regex will change from True to False in a future version.
/home/jovyan/data-analyses/project_prioritization/_sb1_utils.py:28: FutureWarning: The default value of regex will change from True to False in a future version.
/home/jovyan/data-analyses/project_prioritization/_sb1_utils.py:42: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal

In [6]:
# nine_sample_projects[['project_name','project_description','county','previous_caltrans_nominations']]

In [7]:
m1 = sb1_utils.m_nonshopp_sb1_project_titles(nonshopp, sb1)

In [8]:
m2 = sb1_utils.m_nonshopp_sb1_fuzzy(m1, nonshopp, sb1)

### Max row limit signifier
* Test with a few dataframes

In [9]:
# Test with TIRCP
tircp = to_snakecase(gpd.read_file("https://odpsvcs.dot.ca.gov/arcgis/rest/services/RCA/RCA_Projects_032022/FeatureServer/5/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=*+&returnGeometry=true&maxAllowableOffset=&geometryPrecision=&outSR=&gdbVersion=&historicMoment=&returnDistinctValues=false&returnIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&multipatchOption=&resultOffset=&resultRecordCount=&returnTrueCurves=false&sqlFormat=none&f=geojson"))

In [10]:
atp = to_snakecase(gpd.read_file("https://odpsvcs.dot.ca.gov/arcgis/rest/services/RCA/RCA_Projects_032022/FeatureServer/12/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=*+&returnGeometry=true&maxAllowableOffset=&geometryPrecision=&outSR=&gdbVersion=&historicMoment=&returnDistinctValues=false&returnIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&multipatchOption=&resultOffset=&resultRecordCount=&returnTrueCurves=false&sqlFormat=none&f=geojson"))

In [11]:
congested_corridors = to_snakecase(gpd.read_file("https://odpsvcs.dot.ca.gov/arcgis/rest/services/RCA/RCA_Projects_032022/FeatureServer/1/query?where=1%3D1&objectIds=&time=&geometry=&geometryType=esriGeometryEnvelope&inSR=&spatialRel=esriSpatialRelIntersects&distance=&units=esriSRUnit_Foot&relationParam=&outFields=*+&returnGeometry=true&maxAllowableOffset=&geometryPrecision=&outSR=&gdbVersion=&historicMoment=&returnDistinctValues=false&returnIdsOnly=false&returnCountOnly=false&returnExtentOnly=false&orderByFields=&groupByFieldsForStatistics=&outStatistics=&returnZ=false&returnM=false&multipatchOption=&resultOffset=&resultRecordCount=&returnTrueCurves=false&sqlFormat=none&f=geojson"))

In [12]:
for i in [tircp,atp,congested_corridors]:
    print(sb1_utils.row_limit(i, 96))

96 rows -> over max limit
321 rows -> over max limit
12 rows -> under max limit


In [13]:
for i in [tircp,atp,congested_corridors]:
    print(sb1_utils.row_limit(i))

96 rows -> under max limit
321 rows -> under max limit
12 rows -> under max limit


In [14]:
# Subset to preview SB1 vs Nonshopp. Nonshopp is on the left, sb1 on the right
preview_cols =  ['project_name','projecttitle','project_description','projectdescription', 'full_county_name', 'countynames']

### Merge Non-Shopp with SB1 - Fuzzy Matches

##### Try with project description since titles are very vague.

In [ ]:
def replace_matches_set_ratio(df, column, new_col_name, string_to_match, min_ratio):
    # Get a list of unique strings
    strings = df[column].unique()

    # Get the top 10 closest matches to our input string
    matches = fuzzywuzzy.process.extract(
        string_to_match, strings, limit=10, scorer=fuzzywuzzy.fuzz.token_set_ratio
    )

    # Only get matches with a  min ratio
    close_matches = [matches[0] for matches in matches if matches[1] > min_ratio]

    # Get the rows of all the close matches in our dataframe
    rows_with_matches = df[column].isin(close_matches)

    # replace all rows with close matches with the input matches
    df.loc[rows_with_matches, new_col_name] = string_to_match

In [ ]:
# Only include projects with a long enough desc
# Drop dups
sb1_w_projectdesc = ((m3.loc[m3.projectdescription_count > 10])
.drop_duplicates(subset = ["projecttitle_x", "projectdescription", "countynames"])
.reset_index(drop = True)
                    )

In [ ]:
# Test with nonshopp 
nonshopp_with_desc = ((nonshopp2.loc[nonshopp2.project_description_count > 10])
                     .drop_duplicates(subset = ["project_name", "project_description", "full_county_name"]).reset_index(drop = True)
                    )

In [ ]:
# Reverse -> replace 
#for i in sb1_w_projectdesc["projectdescription"].loc[1001:3000].unique().tolist():
#    replace_matches_set_ratio(
#        nonshopp_with_desc, "project_description", "project_desc_fuzzy_match", i, 95
#  )

In [ ]:
# nonshopp_with_desc = nonshopp_with_desc.loc[nonshopp_with_desc.project_desc_fuzzy_match.notnull()].reset_index(drop = True)

In [ ]:
# len(nonshopp_with_desc)

In [ ]:
#nonshopp_with_desc[['project_description','project_desc_fuzzy_match']]

In [ ]:
# fuzzy_desc_list = nonshopp_with_desc.project_description.unique().tolist()

In [ ]:
fuzzy_desc_list = ['in humboldt county on us 101 between the 6th street interchange pm 984 and the trinidadmain street interchange pm 1007 and along scenic drive and westhaven drive between the community of westhaven and trinidad construction of a new interchange on us 101 and new road connections from us 101 to scenic drive and westhaven drive associated pedestrian and bicycle improvements through the new interchange and along scenic drive and westhaven drive other work will include associated grading drainage utility relocations signage and striping retaining walls etckim floyd 707 4415739',
 'on route 70 from laurellen to yubabutte county line continuous passing lanes\n\nconstruction 192 of passing lane miles on the state highway',
 'eastbound sr12 to eastbound i80 connector  this project would construct a new connector from eastbound sr12 to eastbound i80 which will also connect two previous cmia projects icp and jameson canyon widening projects reconstruction of the i80i680sr12 interchange complex phase 1 includes remaining packages 3 7\n\nmodifyimprovereconstruct 1 interchange and 1 bridge and construct 1 new bridge',
 'on state route 68 from josselyn canyon road to spreckels blvd operational improvements replaces signalized intersections with roundabouts to achieve smooth traffic flow provide active transportation facilities at intersections and achieve transit benefits achieving these improvements on the existing corridor will preempt need for previouslyconsidered bypass alignment',
 'in fresno and kings county about 6 miles north of lemoore from 03 mile north of excelsior avenue undercrossing to 10 mile north of elkhorn avenue widen from 2lane conventional highway to 4lane expressway this project will eliminate the only remaining bottleneck 2lane section of sr 41 between fresno and lemoore on this interregional route between the san joaquin valley and the central coast and reduce accident rates\n\nconstruct 24 miles of mixed flow lanes 24 miles of mixed flow mainline 8 new curb ramps and 2 intersectionsignal improvements',
 'in fresno county in and near fresno from 04 mile south of american avenue to 04 mile north of north avenue modify interchanges',
 'in and near lost hills from 1 mile west of browns material road to california aquaduct  convert from 2lane conventional highway to 4lane expressway\n\nnew roadway lane miles 69',
 'in diamond bar and the city of industry on route 60 from eb 60 to sb 57 connector overcrossing to near golden springs drive undercrossing and route 57 from nb 57 to wb 60 connector overcrossing to south 5760 separation interchange modifications including auxiliary lanes and three new bridges\n\npse completion target date  22822\n4 interchange modifications 15 miles of auxilary lanes 163929 sqft of new bridgestunnels',
 'add one eastbound auxiliary lane 086 miles on sr91 in the city of long beach on sr91 from i710 to cherry avenue undercrossing the proposed improvements consist of adding one auxiliary lane in the eastbound direction extending the outside 5 lane beyond the atlantic ave eb offramp to cherry ave then dropping it before the cherry ave undercrossing and widening the orange ave and walnut ave undercrossing',
 'replace eastbound rte 91 to northbound rte 71 loop connector with a direct connector ramp realign the green river road eastbound entrance ramp to rte 91 and construct a collectordistributor system on rte 91 in the eastbound direction between the green river road and serfas club drive\n1  traffic monitoring detection station\n1  interchange modification\n1 mile  aux lane constructed\n1  freeway ramp meter\n10000 sqft  new local road bridge structuretunnel\n1  grade separation  rail crossing improvements\n2  new curb ramp installed\n05 miles  aux lanes\n2000 lf  culvert\n25 miles  ramp and connector constructed\n2000000 sqft  modifiedimproved interchange\n1  ramp modification',
 'realign approximately 3 miles of a segment of sr 4 from west of hunt road to appaloosa way beginning approximately 38 miles east of copperopolis and ending approximately 52 miles west of the sr 4sr 49 junction in altaville calaveras county\n\nproject will provide 5 curve and vertical alignment corrections construct 7 turn pocket to improve operations construct 624 miles of mainline shoulders and construct 642 miles of new roadway lane miles',
 'located in the city of tracy at the i580 and international parkwaypatterson pass rd interchange the project proposes to modify the interchange from the existing compact diamond type l1 to a diverging diamond interchange ddi to accommodate increased commercial truck and traffic demand from existing fulfillment distribution center growth and planned future development in san joaquin county the project will improve interregional goods movement for trucks traveling between the port of oakland bay area local warehouses southern california and outofstate while reducing congestion increases truck throughput and eliminate bottleneck for trucks accessing i580 a key freight highway\n\nprovide 1 interchange modifications 2 intersectionsignal improvements 056 miles of communications fiber optics 14 miles of bikeped facilities and modifyimprove 18680 sqft of interchange']

In [ ]:
nonshopp_sb1_m3 = pd.merge(
    m3,
    nonshopp_with_desc[nonshopp_with_desc.project_description.isin(fuzzy_desc_list)],
    how="inner",
    left_on=["projectdescription", "countynames"],
    right_on=["project_desc_fuzzy_match", "full_county_name"],)

In [ ]:
len(nonshopp_sb1_m3)

In [ ]:
type(nonshopp_sb1_m3)

In [ ]:
# nonshopp_sb1_m3

### Compare with 9 Sample Projects

In [ ]:
nine_projects_id = [
    "0422000202",
    "0414000032",
    "0520000083",
    "0515000063",
    "0721000056",
    "0716000370",
    "0813000222",
    "0814000144",
    "0414000032",
    "0720000165",
]

In [ ]:
nine_sample_projects = (nonshopp2[nonshopp2.ct_project_id.isin(nine_projects_id)].reset_index(drop=True))

In [ ]:
len(nine_sample_projects)

In [ ]:
# Subset sb1_geo to only programs these 9 projects have applied for
tcep_sccp1 = m3[m3["programcodes"].str.contains(('TCEP|SCCP'))].reset_index(drop = True)

In [ ]:
# Subset sb1_geo to only programs these 9 projects are located in
tcep_sccp2 = m3[m3["countynames"].str.contains(('Alameda|San Mateo|Santa Cruz|San Luis Obispo|Los Angeles|San Bernardino|Riverside'))].reset_index(drop = True)

In [ ]:
tcep_sccp2.shape

In [ ]:
tcep_sccp2.programcodes.value_counts()

In [ ]:
tcep_sccp2.columns

In [ ]:
pd.merge(
    nine_sample_projects,
    tcep_sccp2.drop(columns = ["_merge"]),
    how="outer",
    left_on=["ct_project_id"],
    right_on=["projectid"],
    indicator=True,
)[["_merge"]].value_counts()

In [ ]:
pd.merge(
    nine_sample_projects,
    tcep_sccp2.drop(columns = ["_merge"]),
    how="outer",
    left_on=["project_name"],
    right_on=["projecttitle_x"],
    indicator=True,
)[["_merge"]].value_counts()

* Eyeballing matches
    * route 395 widening from sr 18 to chamberlaine way in SB1 could match us 395 freight mobility and safety project in Non SHOPP
    * state route 1 state park to bayporter auxiliary lanes in SB1 is watsonvillesanta cruz multimodal corridor program wscmcp cycle 3 project contract 1 sr 1 freedom to state park aux lanes bus on shoulders and coastal rail trail segment 12 in non SHOPP

In [ ]:
# tcep_sccp2[['projecttitle_x','agencies','countynames', 'projectdescription']].sort_values(['countynames','projecttitle_x'])

In [ ]:
# nine_sample_projects[['project_name','lead_agency','full_county_name','project_description']].sort_values(['full_county_name','project_name'])

In [ ]:
non_shopp_projects_sb1_list = ['route 395 widening from sr 18 to chamberlaine way', 
                             'state route 1  state park to bayporter auxiliary lanes']

In [ ]:
non_shopp_projects_in_sb1 = tcep_sccp2[tcep_sccp2["projecttitle_x"].isin(non_shopp_projects_sb1_list)].reset_index(drop = True)

In [ ]:
non_shopp_projects_in_sb1.projecttitle_x = non_shopp_projects_in_sb1.projecttitle_x.replace({
    'route 395 widening from sr 18 to chamberlaine way': 'us 395 freight mobility and safety project',
    'state route 1  state park to bayporter auxiliary lanes': 'watsonvillesanta cruz multimodal corridor program wscmcp cycle 3 project contract 1  sr 1 freedom to state park aux lanes bus on shoulders and coastal rail trail segment 12'
})

In [ ]:
non_shopp_projects_in_sb1[['agencies','programcodes', 'countynames','projecttitle_x','projectdescription']]

In [ ]:
nonshopp_sb1_m4 = pd.merge(
    non_shopp_projects_in_sb1,
    nine_sample_projects,
    how="inner",
    left_on=["projecttitle_x"],
    right_on=["project_name"],
)

In [ ]:
len(nonshopp_sb1_m4)

In [ ]:
# nine_sample_projects_geo[['project_name','projecttitle','project_description','projectdescription', 'full_county_name']]

In [ ]:
#  _utils.geojson_gcs_export(nine_sample_projects_geo,_utils.GCS_FILE_PATH, 'nine_sample_projects_geom')

### Concat all the matches

In [ ]:
nonshopp_geo = pd.concat([nonshopp_sb1_m1, nonshopp_sb1_m2, nonshopp_sb1_m3, nonshopp_sb1_m4], sort=False)

In [ ]:
type(nonshopp_geo)

In [ ]:
len(nonshopp_geo)

In [ ]:
sb1_cols = m3.columns.tolist()

In [ ]:
sb1_cols.remove('geometry')

In [ ]:
sb1_cols.extend(['project_name_count',
 'project_description_count',
 'project_title_fuzzy_match',
 'project_desc_fuzzy_match'])

In [ ]:
# Delete out the columns from SB1 except geometry
nonshopp_geo2 = nonshopp_geo.drop(columns = sb1_cols)

In [ ]:
nonshopp_geo2[['previous_caltrans_nominations']].drop_duplicates()

In [ ]:
# nonshopp_geo2.explore(tooltip = ['project_name','project_description', 'lead_agency', 'previous_caltrans_nominations'], marker_type='marker', marker_kwds={"radius":3})

In [ ]:
# utils.geoparquet_gcs_export(nonshopp_geo2,_utils.GCS_FILE_PATH, 'nonshopp_gdf')

### Save as Geoparquet

In [ ]:
#with get_fs().open(f"{_utils.GCS_FILE_PATH}nine_sample_projects_geom.geojson") as f:
 #   nine_projects = to_snakecase(gpd.read_file(f))

In [ ]:
# nine_projects

In [ ]:
# utils.geoparquet_gcs_export(nine_projects, _utils.GCS_FILE_PATH, "nine_sample_projects")

In [ ]:
#with get_fs().open(f"{_utils.GCS_FILE_PATH}nonshopp_gdf.geojson") as f:
#    nonshopp_with_geo = to_snakecase(gpd.read_file(f))

In [ ]:
# utils.geoparquet_gcs_export(nonshopp_with_geo, _utils.GCS_FILE_PATH, "nonshopp_with_geo")

In [ ]:
#test1 = gpd.read_parquet(
#    "gs://calitp-analytics-data/data-analyses/project_prioritization/nine_sample_projects.parquet"
#)

In [ ]:
#test2 = gpd.read_parquet(
#    "gs://calitp-analytics-data/data-analyses/project_prioritization/nonshopp_with_geo.parquet"
#)

In [ ]:
# test2.explore()